<a href="https://colab.research.google.com/github/Priyanshu-Shah/ml_practice/blob/main/model_using_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# class Model(nn.Module):

#   def __init__(self, num_features):

#     super().__init__()

#     self.network = nn.Sequential(
#         nn.Linear(num_features, 3),
#         nn.ReLU(),
#         nn.Linear(3, 1),
#         nn.Sigmoid()
#     )

#   def forward(self, features):
#     #used sequential container so that every var is not needed to be forward passed
#     out = self.network(features)
#     return out

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [11]:
#importing the dataset from guthyub using pandas
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [12]:
#dropping unwanted collumns
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [13]:
#splitting the dataset into training and testing splits
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,1:], df.iloc[:, 0], test_size =  0.2) #size 0.2 means 20-80% split between train and test
#X has all rows, and all collums from radius_mean to end
#Y has all rows, and collum diagnosis i.e collum 0 after dropping id

In [14]:
#standardise the features to fit in the model
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #fit calculates the mean and std dev of x_train and scales it s.t. mean is at 0 and std dev is 1
X_test = scaler.transform(X_test) #transform the test data in the same way

In [15]:
Y_train

,diagnosis
540,B
369,M
213,M
345,B
459,B
...,...
3,M
49,B
146,M
53,M


In [16]:
#convert Y_train's labels into numerical values so it can be used by the model
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

In [23]:
#convert the np arrays into pytorch tensors
# Explicitly set the dtype to float32
X_train_tensor = torch.from_numpy(X_train).type(torch.float32)
X_test_tensor = torch.from_numpy(X_test).type(torch.float32)
Y_train_tensor = torch.from_numpy(Y_train).type(torch.float32)
Y_test_tensor = torch.from_numpy(Y_test).type(torch.float32)

In [36]:
#we are using a single neuron and 30 features i.e. 30 weights and 1 bias
class NN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out


In [44]:
learning_rate = 0.1
epochs = 25

In [45]:
loss_func = nn.BCELoss()

In [47]:
model = NN(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


for epoch in range(epochs):
  #forward pass
  Y_pred = model(X_train_tensor)

  #calculate loss
  loss = loss_func(Y_pred, Y_train_tensor.view(-1,1))

  #clear gradients
  optimizer.zero_grad()

  #backward pass
  loss.backward()

  #update params
  optimizer.step()

  print(f'Epoch: {epoch+1}, loss: {loss.item()}')

Epoch: 1, loss: 0.8664975166320801
Epoch: 2, loss: 0.6283291578292847
Epoch: 3, loss: 0.5071024894714355
Epoch: 4, loss: 0.4353545904159546
Epoch: 5, loss: 0.3872039318084717
Epoch: 6, loss: 0.3521805703639984
Epoch: 7, loss: 0.32527777552604675
Epoch: 8, loss: 0.3037889897823334
Epoch: 9, loss: 0.28611525893211365
Epoch: 10, loss: 0.27124711871147156
Epoch: 11, loss: 0.25851303339004517
Epoch: 12, loss: 0.24744734168052673
Epoch: 13, loss: 0.2377156913280487
Epoch: 14, loss: 0.22907118499279022
Epoch: 15, loss: 0.22132675349712372
Epoch: 16, loss: 0.21433794498443604
Epoch: 17, loss: 0.20799091458320618
Epoch: 18, loss: 0.20219460129737854
Epoch: 19, loss: 0.19687515497207642
Epoch: 20, loss: 0.19197191298007965
Epoch: 21, loss: 0.18743446469306946
Epoch: 22, loss: 0.18322056531906128
Epoch: 23, loss: 0.17929454147815704
Epoch: 24, loss: 0.17562584578990936
Epoch: 25, loss: 0.17218835651874542


In [48]:
#evaluation
with torch.no_grad():
  Y_pred = model(X_test_tensor)
  Y_pred = (Y_pred > 0.8).float()
  accuracy = (Y_pred == Y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5166205167770386
